<a href="https://colab.research.google.com/github/ahmed-gharib89/TensorFlow_2_for_Deep_Learning/blob/master/Getting%20started%20with%20TensorFlow%202/week4/Saving_model_architecture_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Saving model architecture only

In this reading you will learn how to save a model's architecture, but not its weights.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import json
import numpy as np

In previous videos and notebooks you have have learned how to save a model's weights, as well as the entire model - weights and architecture.

### Accessing a model's configuration
A model's *configuration* refers to its architecture. TensorFlow has a convenient way to retrieve a model's architecture as a dictionary. We start by creating a simple fully connected feedforward neural network with 1 hidden layer.

In [2]:
model = Sequential([
    Dense(units=32, input_shape=(32, 32, 3), activation='relu', name='dense_1'),
    Dense(units=10, activation='softmax', name='dense_2')
])

A TensorFlow model has an inbuilt method `get_config` which returns the model's architecture as a dictionary:

In [3]:
config_dict = model.get_config()
print(config_dict)

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 32, 32, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'dense_1_input'}}, {'class_name': 'Dense', 'config': {'name': 'dense_1', 'trainable': True, 'batch_input_shape': (None, 32, 32, 3), 'dtype': 'float32', 'units': 32, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}, {'class_name': 'Dense', 'config': {'name': 'dense_2', 'trainable': True, 'dtype': 'float32', 'units': 10, 'activation': 'softmax', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None

In [4]:
from pprint import pprint

pprint(config_dict)

{'layers': [{'class_name': 'InputLayer',
             'config': {'batch_input_shape': (None, 32, 32, 3),
                        'dtype': 'float32',
                        'name': 'dense_1_input',
                        'ragged': False,
                        'sparse': False}},
            {'class_name': 'Dense',
             'config': {'activation': 'relu',
                        'activity_regularizer': None,
                        'batch_input_shape': (None, 32, 32, 3),
                        'bias_constraint': None,
                        'bias_initializer': {'class_name': 'Zeros',
                                             'config': {}},
                        'bias_regularizer': None,
                        'dtype': 'float32',
                        'kernel_constraint': None,
                        'kernel_initializer': {'class_name': 'GlorotUniform',
                                               'config': {'seed': None}},
                        'kernel_regularizer'

### Creating a new model from the config
A new TensorFlow model can be created from this config dictionary. This model will have reinitialized weights, which are not the same as the original model.

In [5]:
model_same_config = tf.keras.Sequential.from_config(config_dict)

We can check explicitly that the config of both models is the same, but the weights are not: 

In [6]:
print('Same config:', 
      model.get_config() == model_same_config.get_config())
print('Same value for first weight matrix:', 
      np.allclose(model.weights[0].numpy(), model_same_config.weights[0].numpy()))

Same config: True
Same value for first weight matrix: False


For models that are not `Sequential` models, use `tf.keras.Model.from_config` instead of `tf.keras.Sequential.from_config`.

### Other file formats: JSON and YAML
It is also possible to obtain a model's config in JSON or YAML formats. This follows the same pattern:

In [8]:
json_string = model.to_json()
pprint(json_string)

('{"class_name": "Sequential", "config": {"name": "sequential", "layers": '
 '[{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32, 32, '
 '3], "dtype": "float32", "sparse": false, "ragged": false, "name": '
 '"dense_1_input"}}, {"class_name": "Dense", "config": {"name": "dense_1", '
 '"trainable": true, "batch_input_shape": [null, 32, 32, 3], "dtype": '
 '"float32", "units": 32, "activation": "relu", "use_bias": true, '
 '"kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": '
 'null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, '
 '"kernel_regularizer": null, "bias_regularizer": null, '
 '"activity_regularizer": null, "kernel_constraint": null, "bias_constraint": '
 'null}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": '
 'true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": '
 'true, "kernel_initializer": {"class_name": "GlorotUniform", "config": '
 '{"seed": null}}, "bias_initializer"

The JSON format can easily be written out and saved as a file:

In [9]:
# Write out JSON config file

with open('config.json', 'w') as f:
    json.dump(json_string, f)
del json_string

In [10]:
# Read in JSON config file again

with open('config.json', 'r') as f:
    json_string = json.load(f)

In [11]:
# Reinitialize model

model_same_config = tf.keras.models.model_from_json(json_string)

In [12]:
# Same config, new weights

print('Same config:', 
      model.get_config() == model_same_config.get_config())
print('Same value for first weight matrix:', 
      np.allclose(model.weights[0].numpy(), model_same_config.weights[0].numpy()))

Same config: True
Same value for first weight matrix: False


The YAML format is similar. The details of writing out YAML files, loading them and using them to create a new model are similar as for the JSON files, so we won't show it here.

In [13]:
yaml_string = model.to_yaml()
pprint(yaml_string)

('backend: tensorflow\n'
 'class_name: Sequential\n'
 'config:\n'
 '  layers:\n'
 '  - class_name: InputLayer\n'
 '    config:\n'
 '      batch_input_shape: !!python/tuple [null, 32, 32, 3]\n'
 '      dtype: float32\n'
 '      name: dense_1_input\n'
 '      ragged: false\n'
 '      sparse: false\n'
 '  - class_name: Dense\n'
 '    config:\n'
 '      activation: relu\n'
 '      activity_regularizer: null\n'
 '      batch_input_shape: !!python/tuple [null, 32, 32, 3]\n'
 '      bias_constraint: null\n'
 '      bias_initializer:\n'
 '        class_name: Zeros\n'
 '        config: {}\n'
 '      bias_regularizer: null\n'
 '      dtype: float32\n'
 '      kernel_constraint: null\n'
 '      kernel_initializer:\n'
 '        class_name: GlorotUniform\n'
 '        config: {seed: null}\n'
 '      kernel_regularizer: null\n'
 '      name: dense_1\n'
 '      trainable: true\n'
 '      units: 32\n'
 '      use_bias: true\n'
 '  - class_name: Dense\n'
 '    config:\n'
 '      activation: softmax\n'
 

Writing out, reading in and using YAML files to create models is similar to JSON files. 

### Further reading and resources
* https://www.tensorflow.org/guide/keras/save_and_serialize#architecture-only_saving
* https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model